In [1]:
import os
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
# Import transforms from torchaudio and torchvision v2
from torchaudio import transforms as TA_T
from torchvision.transforms import v2 as T
from torchvision import models # Import models submodule
from torchvision.models import EfficientNet_B0_Weights # Import specific weights
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, auc
)
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import warnings
from urllib.request import urlretrieve # Keep for potential future downloads

# --- Setup ---
warnings.simplefilter(action='ignore', category=FutureWarning)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

Using device: cuda


In [2]:
class TimeShift(torch.nn.Module):
    """ Dịch chuyển spectrogram theo trục thời gian một cách ngẫu nhiên. """
    def __init__(self, p=0.5, max_shift_percent=0.4):
        super().__init__()
        self.p = p
        self.max_shift_percent = max_shift_percent

    def forward(self, spec):
        # Input spec shape: (..., Frequency, Time)
        if torch.rand(1) < self.p:
            time_steps = spec.shape[-1]
            shift_amount = int(torch.rand(1).item() * self.max_shift_percent * time_steps) # Use .item()
            if torch.rand(1) < 0.5:
                shift_amount = -shift_amount
            # Apply roll only if shift_amount is non-zero
            if shift_amount != 0:
                 return torch.roll(spec, shifts=shift_amount, dims=-1)
        return spec

class SpecNoise(torch.nn.Module):
    """ Thêm nhiễu Gaussian trực tiếp vào spectrogram. """
    def __init__(self, p=0.5, noise_level=0.05):
        super().__init__()
        self.p = p
        self.noise_level = noise_level

    def forward(self, spec):
        if torch.rand(1) < self.p:
            # Add noise relative to std dev might be better for varying signal levels
            # noise = torch.randn_like(spec) * self.noise_level * torch.std(spec)
            # Keeping original implementation for now:
            noise = torch.randn_like(spec) * self.noise_level
            return spec + noise
        return spec

# --- Transforms for EfficientNetB0 ---
IMG_SIZE = 224 # EfficientNetB0 standard input size
NORM_MEAN = [0.485] # Using first value of ImageNet mean for single channel
NORM_STD = [0.229]  # Using first value of ImageNet std dev for single channel

train_transform_effnet = T.Compose([
    # Input expected: (1, H, W) after dataset adds channel dim
    TA_T.TimeMasking(time_mask_param=80),
    TA_T.FrequencyMasking(freq_mask_param=25),
    TimeShift(p=0.5, max_shift_percent=0.4),
    SpecNoise(p=0.3, noise_level=0.03),
    T.Resize((IMG_SIZE, IMG_SIZE), antialias=True), # Resize to square
    T.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

val_transform_effnet = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE), antialias=True),
    T.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

print(f"Đã định nghĩa pipeline Augmentation cho EfficientNetB0 (input {IMG_SIZE}x{IMG_SIZE}).")

Đã định nghĩa pipeline Augmentation cho EfficientNetB0 (input 224x224).


In [3]:
class EfficientNet_AudioClassifier(nn.Module):
    def __init__(self, num_classes, input_channels=1, pretrained=True):
        super().__init__()
        weights = EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
        self.effnet = models.efficientnet_b0(weights=weights)

        # Adapt the first convolutional layer
        original_first_conv = self.effnet.features[0][0]
        original_weights = None
        if pretrained:
            original_weights = original_first_conv.weight.data

        new_first_conv = nn.Conv2d(
            in_channels=input_channels,
            out_channels=original_first_conv.out_channels,
            kernel_size=original_first_conv.kernel_size,
            stride=original_first_conv.stride,
            padding=original_first_conv.padding,
            bias=original_first_conv.bias is not None
        )

        if pretrained and input_channels != original_first_conv.in_channels:
            print(f"Adapting EfficientNetB0 pretrained weights from {original_first_conv.in_channels} to {input_channels} channels.")
            adapted_weights = original_weights.mean(dim=1, keepdim=True)
            if input_channels > 1:
                 adapted_weights = adapted_weights.repeat(1, input_channels, 1, 1)
            new_first_conv.weight.data = adapted_weights
            if new_first_conv.bias is not None and original_first_conv.bias is not None:
                new_first_conv.bias.data = original_first_conv.bias.data
        elif pretrained: # Channels match
             new_first_conv.load_state_dict(original_first_conv.state_dict())

        self.effnet.features[0][0] = new_first_conv

        # Adapt the classifier
        num_ftrs = self.effnet.classifier[1].in_features
        self.effnet.classifier[1] = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        # Input expected: (B, C, H, W)
        if x.dim() == 3: # Add channel dim if dataset didn't
            x = x.unsqueeze(1)
        # Check channel consistency (should match input_channels used in __init__)
        if x.shape[1] != self.effnet.features[0][0].in_channels:
             raise ValueError(f"Input tensor channel {x.shape[1]} != model first layer channel {self.effnet.features[0][0].in_channels}")
        return self.effnet(x)

print("Đã định nghĩa xong kiến trúc model EfficientNetB0.")

Đã định nghĩa xong kiến trúc model EfficientNetB0.


In [4]:
class SpectrogramDataset(Dataset):
    def __init__(self, metadata_df, melspec_dir, transform=None, target_tdim=626):
        super().__init__()
        self.metadata = metadata_df
        self.melspec_dir = melspec_dir
        self.transform = transform
        # target_tdim is still relevant for padding/truncating *before* resizing
        self.target_tdim = target_tdim
        # Ensure 'label' column exists
        if 'label' not in self.metadata.columns:
            raise ValueError("Metadata DataFrame must contain a 'label' column.")
        self.class_names = sorted(metadata_df['label'].unique())
        self.class_to_id = {name: i for i, name in enumerate(self.class_names)}
        self.error_log = []

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        if idx >= len(self.metadata):
             self.error_log.append(f"Index {idx} out of bounds ({len(self.metadata)})")
             return None
        # Ensure required columns exist
        if 'file_path' not in self.metadata.columns or 'label' not in self.metadata.columns:
             self.error_log.append(f"Missing column at index {idx}")
             return None

        row = self.metadata.iloc[idx]
        relative_path = row['file_path']
        label_name = row['label']

        if label_name not in self.class_to_id:
             self.error_log.append(f"Label '{label_name}' (idx {idx}) not in class map.")
             return None
        label = self.class_to_id[label_name]

        file_path = os.path.join(self.melspec_dir, relative_path)

        if not os.path.exists(file_path):
             self.error_log.append(f"File not found: {file_path}")
             return None

        try:
            loaded_object = torch.load(file_path, map_location=torch.device('cpu'))
            if isinstance(loaded_object, (tuple, list)):
                if len(loaded_object) > 0 and isinstance(loaded_object[0], torch.Tensor):
                    spectrogram = loaded_object[0]
                else: raise TypeError("Loaded tuple/list contains no tensor.")
            elif isinstance(loaded_object, torch.Tensor): spectrogram = loaded_object
            else: raise TypeError(f"Unexpected data type: {type(loaded_object)}")

            spectrogram = spectrogram.to(torch.float32)

            if spectrogram.dim() > 2: spectrogram = spectrogram.squeeze()
            if spectrogram.dim() != 2: raise ValueError(f"Not 2D: {spectrogram.shape}")

            # Pad/Truncate Time Dim *before* resize
            current_tdim = spectrogram.shape[-1]
            if current_tdim > self.target_tdim:
                start = (current_tdim - self.target_tdim) // 2
                spectrogram = spectrogram[..., start:start + self.target_tdim]
            elif current_tdim < self.target_tdim:
                spectrogram = F.pad(spectrogram, (0, self.target_tdim - current_tdim))

            # Add channel dimension: (H, W) -> (1, H, W) for transforms
            spectrogram = spectrogram.unsqueeze(0)

            if self.transform:
                spectrogram = self.transform(spectrogram) # Apply transforms

            # Final shape check (after transforms like Resize)
            # Example: Should be (1, IMG_SIZE, IMG_SIZE) for EfficientNet
            # if spectrogram.shape != (1, IMG_SIZE, IMG_SIZE):
            #    self.error_log.append(f"Unexpected shape after transform {spectrogram.shape} for {file_path}")
            #    return None

            return spectrogram, label
        except Exception as e:
            self.error_log.append(f"Error loading/processing {file_path}: {str(e)}")
            return None

    def get_error_log(self):
        return self.error_log

print("Đã cập nhật xong SpectrogramDataset.")

def collate_fn_remove_none(batch):
    batch = [item for item in batch if item is not None and item[0] is not None]
    if not batch:
        return torch.tensor([], device=device, dtype=torch.float32), torch.tensor([], device=device, dtype=torch.long)
    try:
        samples, labels = zip(*batch)
        samples_stacked = torch.stack(samples)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        return samples_stacked, labels_tensor
    except RuntimeError as e:
        print(f"Error stacking tensors in collate_fn: {e}")
        # Print shapes for debugging
        for i, s in enumerate(samples): print(f"Sample {i} shape: {s.shape}")
        return torch.tensor([], device=device, dtype=torch.float32), torch.tensor([], device=device, dtype=torch.long)
    except Exception as e_gen:
        print(f"Generic error in collate_fn: {e_gen}")
        return torch.tensor([], device=device, dtype=torch.float32), torch.tensor([], device=device, dtype=torch.long)

Đã cập nhật xong SpectrogramDataset.


In [5]:
def plot_roc_curve(val_labels, val_probs, class_names, save_path):
    n_classes = len(class_names)
    if n_classes <= 1:
        print("ROC plotting skipped: <= 1 class.")
        return
    val_labels_np = np.array(val_labels)
    val_probs_np = np.array(val_probs)
    if val_probs_np.shape[1] != n_classes:
         print(f"Warning: Prob columns ({val_probs_np.shape[1]}) != classes ({n_classes}). ROC incorrect.")
    fpr, tpr, roc_auc = dict(), dict(), dict()
    try:
        val_labels_one_hot = F.one_hot(torch.tensor(val_labels_np, dtype=torch.long), num_classes=n_classes).numpy()
    except Exception as e:
        print(f"Error creating one-hot labels: {e}. Skipping ROC plot."); return
    plt.figure(figsize=(10, 8))
    for i in range(n_classes):
        try:
            fpr[i], tpr[i], _ = roc_curve(val_labels_one_hot[:, i], val_probs_np[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc[i]:.4f})')
        except ValueError as e:
            print(f"ROC Calc Warn (Class {class_names[i]}): {e}"); roc_auc[i] = float('nan')
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')
    plt.xlim([0.0, 1.0]); plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve (One-vs-Rest)'); plt.legend(loc="lower right"); plt.grid(True)
    try: plt.savefig(save_path)
    except Exception as e: print(f"ROC Save Err: {e}")
    plt.close()

# --- Training and Evaluation Function ---
def train_and_evaluate( model, train_loader, val_loader, epochs, results_dir, run_base_name, class_names ):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5) # Adjusted LR/WD
    criterion = nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.2, patience=5, verbose=True) # Verbose scheduler
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
    best_f1_weighted = 0.0
    best_metrics_dict = {}
    os.makedirs(results_dir, exist_ok=True)
    metrics_csv_path = os.path.join(results_dir, f"metrics_history_{run_base_name}.csv")
    best_checkpoint_path = os.path.join(results_dir, f"Best_{run_base_name}.pth")
    report_path = os.path.join(results_dir, f"Best_{run_base_name}_report.txt")
    cm_path = os.path.join(results_dir, f"Best_{run_base_name}_confusion_matrix.csv")
    auc_plot_path = os.path.join(results_dir, f"Best_{run_base_name}_AUC_ROC.png")
    csv_header = ['Epoch', 'LR', 'Train_Loss', 'Accuracy', 'Precision_micro', 'Precision_macro', 'Precision_w', 'Recall_micro', 'Recall_macro', 'Recall_w', 'F1_micro', 'F1_macro', 'F1_w', 'AUC_macro_ovr']
    if not os.path.exists(metrics_csv_path):
        pd.DataFrame(columns=csv_header).to_csv(metrics_csv_path, index=False)

    print(f"\nStarting training for {run_base_name}...")
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0; num_train_batches = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for inputs, labels in loop:
            if inputs.shape[0] == 0: continue
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item(); num_train_batches += 1
            loop.set_postfix(loss=loss.item())
        avg_train_loss = train_loss / num_train_batches if num_train_batches > 0 else 0
        current_lr = optimizer.param_groups[0]['lr']

        model.eval()
        val_preds, val_labels, val_probs = [], [], []
        val_loop = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        with torch.no_grad():
            for inputs, labels in val_loop:
                if inputs.shape[0] == 0: continue
                inputs, labels = inputs.to(device), labels.to(device)
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                    outputs = model(inputs)
                probs = F.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1) # Use outputs directly
                val_preds.extend(predicted.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                val_probs.extend(probs.cpu().numpy())

        if not val_labels:
            print(f"Epoch {epoch+1}: No validation samples processed. Skipping metrics.")
            continue
        val_labels_np = np.array(val_labels); val_preds_np = np.array(val_preds); val_probs_np = np.array(val_probs)
        metrics_data = {'Epoch': epoch + 1, 'LR': current_lr, 'Train_Loss': avg_train_loss}
        try:
             metrics_data['Accuracy'] = accuracy_score(val_labels_np, val_preds_np)
             metrics_data['Precision_micro'] = precision_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['Precision_macro'] = precision_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['Precision_w'] = precision_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             metrics_data['Recall_micro'] = recall_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['Recall_macro'] = recall_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['Recall_w'] = recall_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             metrics_data['F1_micro'] = f1_score(val_labels_np, val_preds_np, average='micro', zero_division=0)
             metrics_data['F1_macro'] = f1_score(val_labels_np, val_preds_np, average='macro', zero_division=0)
             metrics_data['F1_w'] = f1_score(val_labels_np, val_preds_np, average='weighted', zero_division=0)
             if len(np.unique(val_labels_np)) > 1 and val_probs_np.shape[1] == len(class_names):
                try: metrics_data['AUC_macro_ovr'] = roc_auc_score(val_labels_np, val_probs_np, multi_class='ovr', average='macro')
                except ValueError as auc_error: metrics_data['AUC_macro_ovr'] = 0.0; print(f"AUC Calc Warn: {auc_error}")
             else: metrics_data['AUC_macro_ovr'] = 0.0
        except Exception as metric_error: print(f"Metric Calc Err: {metric_error}"); continue
        try:
             ordered_metrics = {h: metrics_data.get(h, None) for h in csv_header}
             pd.DataFrame([ordered_metrics]).to_csv(metrics_csv_path, mode='a', header=False, index=False)
        except Exception as csv_error: print(f"CSV Write Err: {csv_error}")

        current_f1_weighted = metrics_data.get('F1_w', 0.0)
        print(f"Epoch {epoch+1}: TrainLoss={avg_train_loss:.4f}, F1-W={current_f1_weighted:.4f}, Acc={metrics_data.get('Accuracy', 0.0):.4f}, AUC={metrics_data.get('AUC_macro_ovr', 0.0):.4f}")

        if current_f1_weighted > best_f1_weighted:
            best_f1_weighted = current_f1_weighted
            best_metrics_dict = metrics_data.copy()
            print(f"  -> New best F1-W: {best_f1_weighted:.4f}. Saving artifacts.")
            try: torch.save(model.state_dict(), best_checkpoint_path)
            except Exception as e: print(f"Save Err: {e}")
            try:
                report = classification_report(val_labels_np, val_preds_np, target_names=class_names, digits=4, zero_division=0)
                with open(report_path, 'w', encoding='utf-8') as f:
                    f.write(f"--- Best Metrics (Epoch {best_metrics_dict.get('Epoch', 'N/A')}) ---\n")
                    for k, v in best_metrics_dict.items():
                        if k not in ['Epoch', 'LR', 'Train_Loss']: f.write(f"{k}: {v:.4f}\n")
                    f.write("\n--- Classification Report ---\n"); f.write(report)
            except Exception as e: print(f"Report Save Err: {e}")
            try:
                cm_df = pd.DataFrame(confusion_matrix(val_labels_np, val_preds_np), index=class_names, columns=class_names)
                cm_df.to_csv(cm_path)
            except Exception as e: print(f"CM Save Err: {e}")
            if metrics_data.get('AUC_macro_ovr', 0.0) > 0.0:
                try: plot_roc_curve(val_labels_np, val_probs_np, class_names, auc_plot_path)
                except Exception as e: print(f"ROC Plot Err: {e}")
        scheduler.step(current_f1_weighted)

    print(f"\nFinished Training for {run_base_name}. Best F1-W: {best_f1_weighted:.4f} at Epoch {best_metrics_dict.get('Epoch', 'N/A')}")
    # Reporting errors after full run for this percentage
    train_errors = train_loader.dataset.get_error_log()
    if train_errors:
         print(f"\n--- Train Dataset Errors ({len(train_errors)} files) for {run_base_name} ---")
         [print(err) for err in train_errors[:min(10, len(train_errors))]]; print("..." if len(train_errors)>10 else "")
    # Validation errors only need to be reported once if test_loader is reused
    # val_errors = val_loader.dataset.get_error_log() ...
    return best_metrics_dict

In [6]:
MELSPEC_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\Finetune_MelSpec'
METADATA_PATH = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\metadata_Finetune_MelSpec.csv'
BASE_OUTPUT_DIR = r'C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare'
print(f"Tất cả kết quả sẽ được lưu vào các thư mục con trong: {BASE_OUTPUT_DIR}")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)

DATA_PERCENTAGES = [0.25, 0.5, 1.0] # Hoặc [1.0] để chạy nhanh
results = {}

Tất cả kết quả sẽ được lưu vào các thư mục con trong: C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare


In [7]:
print("Preparing dataset...")
try:
    full_metadata = pd.read_csv(METADATA_PATH)
except FileNotFoundError: print(f"Error: Metadata file not found: {METADATA_PATH}"); exit()
if 'file_path' not in full_metadata.columns or 'label' not in full_metadata.columns:
    print("Error: Metadata CSV must contain 'file_path' and 'label'."); exit()
initial_len = len(full_metadata)
full_metadata.dropna(subset=['file_path', 'label'], inplace=True)
if len(full_metadata) < initial_len: print(f"Warning: Dropped {initial_len - len(full_metadata)} rows with missing values.")

num_classes = len(full_metadata['label'].unique())
if num_classes == 0: print("Error: No classes found."); exit()
print(f"Number of classes: {num_classes}")

indices = np.arange(len(full_metadata)); labels = full_metadata['label'].values
try:
    train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=SEED, stratify=labels)
    print(f"Data split: {len(train_indices)} train, {len(test_indices)} test.")
except ValueError as e:
     print(f"Stratify Error: {e}. Using non-stratified split."); train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=SEED)
     print(f"Split: {len(train_indices)} train, {len(test_indices)} test.")

# --- Create Test Loader (using val_transform_effnet) ---
test_metadata = full_metadata.iloc[test_indices].copy()
try:
    test_dataset = SpectrogramDataset(metadata_df=test_metadata, melspec_dir=MELSPEC_DIR, transform=val_transform_effnet) # USE EFFNET TRANSFORM
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0, collate_fn=collate_fn_remove_none, pin_memory=torch.cuda.is_available())
    class_names_list = test_dataset.class_names
    print(f"Class names: {class_names_list}")
    print(f"Test dataset size: {len(test_dataset)}")
except ValueError as e: # Catch potential errors from Dataset init
    print(f"Error initializing test dataset: {e}"); exit()
except Exception as e_gen:
    print(f"Unexpected error initializing test dataset: {e_gen}"); exit()

# --- Define Model Configs (using EfficientNetB0) ---
model_configs = {
    "EfficientNetB0": {
        "model_class": EfficientNet_AudioClassifier,
        "args": {"num_classes": num_classes, "input_channels": 1, "pretrained": True}
    }
}

Preparing dataset...
Number of classes: 8
Data split: 11240 train, 2810 test.
Class names: ['Gryllus_bimaculatus', 'Gryllus_campestris', 'Halyomorpha', 'Melolontha_melolontha', 'Myzus', 'Nezara', 'Trialeurodes', 'Tuta']
Test dataset size: 2810


In [8]:
for model_name, config in model_configs.items():
    print(f"\n{'='*20} Evaluating: {model_name} {'='*20}")
    model_results = []
    model_folder_name = model_name.replace(' ', '_').replace('-', '_') # Sanitize name
    model_output_dir = os.path.join(BASE_OUTPUT_DIR, model_folder_name)
    os.makedirs(model_output_dir, exist_ok=True)
    print(f"Saving results for this model to: {model_output_dir}")

    # Loop through data percentages
    for percent in DATA_PERCENTAGES:
        print(f"\n--- Training with {percent*100:.1f}% of labeled data ---")

        # Create subset of training indices
        if percent < 1.0:
             train_labels_for_stratify = full_metadata['label'].iloc[train_indices].values
             min_samples_per_class = pd.Series(train_labels_for_stratify).value_counts().min()
             required_samples = 2
             if min_samples_per_class >= required_samples:
                subset_train_indices, _ = train_test_split(train_indices, train_size=percent, random_state=SEED, stratify=train_labels_for_stratify)
             else:
                 print(f"Warning: Cannot stratify subset (min class samples: {min_samples_per_class}). Using non-stratified subset.")
                 subset_train_indices, _ = train_test_split(train_indices, train_size=percent, random_state=SEED)
        else:
            subset_train_indices = train_indices

        # Create Train Loader (using train_transform_effnet)
        train_metadata_subset = full_metadata.iloc[subset_train_indices].copy()
        try:
            train_dataset = SpectrogramDataset(metadata_df=train_metadata_subset, melspec_dir=MELSPEC_DIR, transform=train_transform_effnet) # USE EFFNET TRANSFORM
            train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, collate_fn=collate_fn_remove_none, pin_memory=torch.cuda.is_available())
            print(f"Train dataset subset size: {len(train_dataset)}")
        except ValueError as e:
             print(f"Error initializing train dataset ({percent*100:.1f}%): {e}"); continue # Skip this percentage
        except Exception as e_gen:
             print(f"Unexpected error initializing train dataset ({percent*100:.1f}%): {e_gen}"); continue

        # Define output paths for this run
        percent_str = f"data_{percent*100:.0f}percent" # Use integer for folder name
        run_output_dir = os.path.join(model_output_dir, percent_str)
        os.makedirs(run_output_dir, exist_ok=True)
        run_base_name = f"{model_folder_name}_{percent_str}"

        # Initialize model
        model = config["model_class"](**config["args"]).to(device)

        # Train and evaluate
        best_metrics = train_and_evaluate(
            model,
            train_loader,
            test_loader, # Use the consistent test_loader for validation
            epochs=100, # Or adjust as needed
            results_dir=run_output_dir,
            run_base_name=run_base_name,
            class_names=class_names_list
        )

        model_results.append({'percent': percent, 'metrics': best_metrics})
        if best_metrics:
            print(f"==> Results with {percent*100:.1f}% data: Best F1-Weighted = {best_metrics.get('F1_w', 0):.4f}")
        else:
            print(f"==> No valid metrics recorded for {percent*100:.1f}% data run.")

    results[model_name] = model_results

    # Report test dataset errors *once* after all percentages for this model
    test_errors = test_loader.dataset.get_error_log()
    if test_errors:
        print(f"\n--- Test Dataset Errors ({len(test_errors)} files) ---")
        [print(err) for err in test_errors[:min(10, len(test_errors))]]; print("..." if len(test_errors)>10 else "")


# --- Final Results Summary ---
print("\n--- FINAL RESULTS (Summary) ---")
for model_name, result_list in results.items():
    print(f"\nModel: {model_name}")
    if not result_list: print("  No results recorded."); continue
    for res in result_list:
        percent_str = f"{res['percent']*100:.1f}% data"
        if res['metrics']:
            f1_w = res['metrics'].get('F1_w', float('nan'))
            f1_m = res['metrics'].get('F1_macro', float('nan'))
            auc = res['metrics'].get('AUC_macro_ovr', float('nan')) # Use correct key
            acc = res['metrics'].get('Accuracy', float('nan'))
            epoch = res['metrics'].get('Epoch', 'N/A')
            print(f"  {percent_str} (Epoch {epoch}): F1-W={f1_w:.4f} | F1-M={f1_m:.4f} | AUC={auc:.4f} | Acc={acc:.4f}")
        else:
            print(f"  {percent_str}: No valid metrics recorded.")


==================== Evaluating: EfficientNetB0 ====================
Saving results for this model to: C:\Users\PC\lavar\audiotwin_checkpoints_medium\Finetune_Dataset_with_mixed_noise_timeshift\SOTA_finetune_compare\EfficientNetB0

--- Training with 25.0% of labeled data ---
Train dataset subset size: 2810
Adapting EfficientNetB0 pretrained weights from 3 to 1 channels.

Starting training for EfficientNetB0_data_25percent...


C:\Users\PC\anaconda3\envs\train_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/100 [Train]:   0%|          | 0/88 [00:00<?, ?it/s]

Epoch 1/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 1: TrainLoss=1.5584, F1-W=0.5434, Acc=0.5779, AUC=0.0000
  -> New best F1-W: 0.5434. Saving artifacts.


...

Epoch 99/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 99: TrainLoss=0.0412, F1-W=0.9099, Acc=0.9100, AUC=0.0000


Epoch 100/100 [Train]:   0%|          | 0/352 [00:00<?, ?it/s]

Epoch 100/100 [Val]:   0%|          | 0/88 [00:00<?, ?it/s]

AUC Calc Warn: Target scores need to be probabilities for multiclass roc_auc, i.e. they should sum up to 1.0 over classes
Epoch 100: TrainLoss=0.0412, F1-W=0.9099, Acc=0.9100, AUC=0.0000

Finished Training for EfficientNetB0_data_100percent. Best F1-W: 0.9131 at Epoch 88
==> Results with 100.0% data: Best F1-Weighted = 0.9131

--- FINAL RESULTS (Summary) ---

Model: EfficientNetB0
  25.0% data (Epoch 38): F1-W=0.8077 | F1-M=0.7862 | AUC=0.0000 | Acc=0.8078
  50.0% data (Epoch 30): F1-W=0.8591 | F1-M=0.8463 | AUC=0.0000 | Acc=0.8594
  100.0% data (Epoch 88): F1-W=0.9131 | F1-M=0.9079 | AUC=0.0000 | Acc=0.9132
